### Hypotheses
- [Lower ripple rate during SD compared NSD](#ripple_rate): One possibility of faster decay of replay during sleep deprivation could be reduced ripple rate at the beginning of POST. This may suggest that in order for cells to replay they need sufficient number/amount of elicitation to last for certain duration of time, so a lower ripple rate could interrupt this requirement.


### Create ripple file backup

In [ ]:
import numpy as np
import subjects

sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday1
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2

    + subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
    + subjects.nsd.ratVday3

)


In [ ]:
for sub,sess in enumerate(sessions):
    sess.ripple.save(sess.filePrefix.with_suffix('.ripples.bak.03-08-2022'))

### Detect ripples for multiple sessions together

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy.plotting import Fig
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday1
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2

    + subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
    + subjects.nsd.ratVday3
)


# sessions = subjects.sd.utkuAG_day1 + subjects.sd.utkuAG_day2


In [ ]:
from tqdm.notebook import tqdm
from neuropy.analyses.oscillations import detect_ripple_epochs

for sub, sess in enumerate(tqdm(sessions)):

    try:
        artifact = sess.artifact
    except:
        artifact = None

    # channels = sess.ripple.metadata['channels'] 
    signal = sess.eegfile.get_signal()
    ripples = detect_ripple_epochs(
        signal, probegroup=sess.probegroup, freq_band=(125, 250), ignore_epochs=artifact
    )

    # ripples.save(sess.filePrefix.with_suffix('.ripple.npy'))
    ripples.save(sess.filePrefix.with_suffix('.ripple.no_merge.npy'))


### Ripple band SNR across channels and depth visualization

In [ ]:
import numpy as np
import subjects

sessions = subjects.sd.ratUday4

In [ ]:
from neuropy.utils.signal_process import hilbertfast,filter_sig

for sub,sess in enumerate(sessions):
    probe_layout = sess.probegroup.to_dataframe()
    probe_layout = probe_layout[probe_layout.connected]
    good_chans = probe_layout.channel_id.values

    snr=[]
    for chan in  good_chans:
        signal = sess.eegfile.get_signal(chan,t_start=0,t_stop=3600)
        rpl_bp = filter_sig.bandpass(signal,lf=125,hf=250).traces[0]
        hilbert_amp = np.abs(hilbertfast(rpl_bp))
        snr.append(np.std(hilbert_amp))


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import Fig

snr = np.asarray(snr)

fig = Fig(grid=(4,2), fontsize=10)
ax = fig.subplot(fig.gs[0])
ax.scatter(probe_layout.x, probe_layout.y, c=snr, cmap="jet")
ax.axis("off")

ax = fig.subplot(fig.gs[1, 0])
ax.plot(good_chans, snr, color="gray")
ax.axhline(np.median(snr), color="k", ls="--")
ax.set_ylabel("SNR")
ax.set_xlabel("Channel id")

ax = fig.subplot(fig.gs[:2, 1])
bins = np.arange(snr.min(), snr.max(), 5)
hist_snr = np.histogram(snr, bins)[0]
ax.plot(bins[:-1] + 2.5, hist_snr, color="gray")
ax.axvline(np.median(snr), color="k", ls="--")


### Detecting noisy ripples using PCA/KMeans
- Since faster-oscillation artifacts have power in ripple band, these may get picked up during ripple detection based on hilbert transform. These ripple-like artifacts can possibly be eliminated using clustering algorithms.  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy.plotting import Fig
import pandas as pd
from scipy import stats
import subjects

sessions = subjects.sd.ratNday1

In [ ]:
from neuropy.utils.signal_process import hilbertfast, filter_sig
from sklearn.decomposition import PCA

for sub, sess in enumerate(sessions):
    good_chans = np.concatenate(sess.probegroup.get_connected_channels())
    pca_ripples = []
    for i in range(len(sess.ripple)):
        rpls = sess.ripple[i].flatten()
        signal = sess.eegfile.get_signal(good_chans.astype("int"), rpls[0], rpls[1])
        hilbert_amp = np.abs(
            hilbertfast(filter_sig.bandpass(signal.traces, lf=130, hf=250, ax=-1), ax=1)
        )

        pca = PCA(n_components=1).fit_transform(hilbert_amp)
        pca_ripples.append(pca.reshape(-1))
pca_ripples = np.asarray(pca_ripples)

In [ ]:
from sklearn.cluster import KMeans,DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import MinMaxScaler
from neuropy.core import Epoch
from sklearn.neighbors import LocalOutlierFactor


pca_of_rpl_amp = PCA(n_components=2).fit_transform(pca_ripples)
features = MinMaxScaler().fit_transform(pca_of_rpl_amp) 
labels = GaussianMixture(n_components=2,random_state=0).fit_predict(features)
outliers= LocalOutlierFactor(n_neighbors=20).fit_predict(features)

# starts,stops = sess.ripple.starts,sess.ripple.stops
# weird_ripples = Epoch.from_array(starts[labels==0],stops[labels==0])
# sess.recinfo.write_epochs(weird_ripples,ext='wrp')


In [ ]:
%matplotlib widget

for l in [0,1]:
    plt.plot(features[labels==l,0],features[labels==l,1],'.')

plt.plot(features[outliers==-1,0],features[outliers==-1,1],'k.')

### Ripple power spectrum comaprison 1st vs 5th vs 8th hour
- To see if ripple lfp timeseries changes PSD during sleep deprivation and compare it to NSD
- Method: Ripple traces are concatenated in their corresponding hour and then a PSD is calculated
- Results: Compared to 1st hour, ripple band in 5th and 8th hour show a little shift towards slower frequencies (maybe the start cutoff for ripple band is only changing)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy.plotting import Fig
import pandas as pd
from tqdm import tqdm
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    +subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2
    +subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
)

rpl_channels = [39,63,111,95,49,100,85,63,63,36,188,16,99,86]

In [ ]:
import scipy.signal as sg
from neuropy.core import Epoch

psd_rpl_df = pd.DataFrame()
norm_psd = lambda p: p / np.sqrt(p)

for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"]
    maze = sess.paradigm["maze"]
    post = sess.paradigm["post"]
    # rpl_chan = sess.ripple.metadata["channels"][2]
    if sess.tag == "sd":
        post = post.flatten()
        post_epochs = Epoch.from_array(
            [post[0], post[0] + 5 * 3600], [post[0] + 5 * 3600, post[1]], ["sd", "rs"]
        )
        all_epochs = pre + maze + post_epochs
    else:
        all_epochs = pre + maze + post

    psd = []
    for e in all_epochs.itertuples():
        signal = sess.eegfile.get_signal(rpl_channels[sub], e.start, e.stop)
        rpl_t = sess.ripple.time_slice(e.start, e.stop).as_array()
        rpl_frames = [np.arange(int(e[0] * 1250), int(e[1] * 1250)) for e in rpl_t]
        rpl_frames = np.concatenate(rpl_frames) - int(e.start * 1250)
        rpl_frames = rpl_frames[rpl_frames < signal.n_frames]
        f, psd = sg.welch(
            signal.traces[0][rpl_frames], fs=1250, nperseg=125, noverlap=62
        )
        psd_rpl_df = psd_rpl_df.append(
            pd.DataFrame(
                {"freq": f, "psd": psd, "Epoch": e.label, "sub": sub, "grp": sess.tag}
            ),
            ignore_index=True,
        )

subjects.GroupData().save(psd_rpl_df, "ripple_psd")


In [ ]:
# %matplotlib widget
import seaborn as sns
from neuropy.plotting import Fig

figure = Fig()
fig, gs = figure.draw(grid=(5, 4))

for i, grp in enumerate(["sd", "nsd"]):
    df = psd_rpl_df[psd_rpl_df["grp"] == grp]
    ax=plt.subplot(gs[i])
    sns.lineplot(
        data=df,
        x="freq",
        y="psd",
        hue="hour",
        ci=None,
        ax=ax,
        legend=None,
    )
    ax.set_yscale('log')
    # ax.set_xscale('log')
    ax.set_xlim([60,350])
    ax.set_ylim([5,100])
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Psd')
    # ax.set_xticks([60,100,200,300],['',100,200,''])
    # ax.set_xticks([100,200],rotation=45)

figure.savefig(subjects.figpath_sd/'ripple_psd_various_epochs')



### Ripple power (z-score) compared NSD vs SD

In [ ]:
import numpy as np
import subjects 
import pandas as pd

sessions = subjects.nsd.ripple_sess + subjects.sd.ripple_sess

In [ ]:
zsc_df = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    post_windows = np.arange(post[0], post[0] + 10 * 3600, 3600)
    post_windows[-1] = np.min([post[1],post_windows[-1]])
    post_epochs = np.vstack([post_windows[:-1], post_windows[1:]]).T
    post_labels = [
        f"{_}-{_+1}" for _ in ((post_epochs[:, 0] - post[0]) / 3600).astype("int")
    ]

    # epochs = np.vstack([pre, maze, post_epochs])
    # epochs_labels = ["pre", "maze"]
    # epochs_labels.extend(post_labels)
    epochs = post_epochs
    epochs_labels = post_labels

    for i, (w, label) in enumerate(zip(epochs, epochs_labels)):
        rpl_df = sess.ripple.time_slice(w[0], w[1]).to_dataframe()
        # rpl_df = rpl_df[rpl_df['peak_frequency']<240]
        starts = rpl_df.start.values
        zscores = rpl_df.peakpower.values

        zsc_df.append(pd.DataFrame(dict(zscore=zscores, t=label,sub=sub, grp=sess.tag)))

zsc_df = pd.concat(zsc_df, ignore_index=True)
# zsc_df = zsc_df[zsc_df.f<240].reset_index(drop=True) # takes care of some boundary effects
# subjects.GroupData().save(zsc_df,'ripple_zscore')

In [ ]:
%matplotlib widget
import seaborn as sns
df = zsc_df[zsc_df['grp']=='SD']
sns.pointplot(data=df,x='t',y='zscore',estimator=np.median,hue='sub')

### Bursty ripples SD vs NSD

In [ ]:
import matplotlib.pyplot as plt
from neuropy.utils.signal_process import WaveletSg
import numpy as np
from neuropy.plotting import Fig
import pandas as pd
from tqdm.notebook import tqdm
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday1
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2

    + subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
    + subjects.nsd.ratVday3
)
rpl_channels = [39,63,111,95,49,117,100,85,63,63,36,188,16,99,86,9]

assert len(rpl_channels)==len(sessions)


In [ ]:
from neuropy.utils.signal_process import filter_sig, hilbertfast
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from neuropy.analyses.brainstates import hmmfit1d

peak_rate = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    zt = np.array([0, 4])
    starts = zt * 3600 + post[0]
    peak_rate_sub = []
    for i, s in enumerate(starts):
        rpls = sess.ripple.time_slice(s, s + 3600)

        srate = sess.eegfile.sampling_rate
        dt = 1 / srate
        smooth_window = 0.008  # 20 ms
        sigma = smooth_window / dt

        rpl_lfp = sess.eegfile.get_frames_within_epochs(rpls, rpl_channels[sub])
        rpl_bp = filter_sig.bandpass(rpl_lfp, fs=1250, lf=125, hf=250)
        hilbert_amp = np.abs(hilbertfast(rpl_bp))

        hilbert_smth = gaussian_filter1d(hilbert_amp, sigma=sigma)
        hilbert_smth_zsc = stats.zscore(hilbert_smth)

        peaks = find_peaks(hilbert_smth_zsc, height=1)[0]
        peak_rate_sub.append(len(peaks) / len(rpls))

    df = pd.DataFrame(dict(zt=zt, peak_rate=peak_rate_sub, grp=sess.tag))
    peak_rate.append(df)

peak_rate = pd.concat(peak_rate, ignore_index=True)


In [ ]:
%matplotlib widget
import seaborn as sns
from neuropy.plotting import Fig

fig = Fig(grid=(3,3),fontsize=12)
ax=fig.subplot(fig.gs[0])
sns.boxplot(
    data=peak_rate, x="zt", y="peak_rate", hue="grp", hue_order=['NSD','SD'],palette=subjects.colors_sd(1.6)
)
sns.swarmplot(
    data=peak_rate,
    x="zt",
    y="peak_rate",
    hue="grp",
    hue_order=['NSD','SD'],
    palette=subjects.colors_sd(1),
    dodge=True,
    size=8,
    # edgecolors='w',
)
ax.set_xticks([0,1],['0-1','4-5'])
ax.set_xlabel('Zt time (h)')
ax.set_ylabel('# hilbert peaks every ripple')
ax.set_title('Amount of ripple merging\nearly vs late')
ax.legend([],frameon=False)
fig.legend(ax,['NSD','SD'],subjects.colors_sd(1.4),x=1,dy=0.06,fontsize=10)


In [ ]:
%matplotlib widget

_,axs= plt.subplots(2,1,sharex=True)
axs[0].plot(rpl_bp)
axs[0].plot(hilbert_amp)
# axs[0].plot(rpl_smth)
axs[0].plot(hilbert_smth,'r')
# axs[0].plot(hilbert_hilbert_amp,'r')
# plt.plot(peaks,hilbert_smth[peaks],'k*')
axs[1].plot(hilbert_smth_zsc)
axs[1].plot(peaks,hilbert_smth_zsc[peaks],'k*')

# axs[1].plot(states)

### Neurons/MUA acg/ccg during ripples compared early to late of POST for SD and NSD
- I observed change in ripple frequency from early to late sleep. One possibility could be change in synchrony from early to late sleep. For example, in early sleep, neurons are firing in sequences during ripples but later most neurons cofire at same time at a little longer time lag. So here I am plotting acg/ccg or similar measure for change in spiketimes synchrony.
- Didn't find anything interesting in pyramidal neurons but for interneurons if CCGs are sorted by their peak time then Zt2 is more similar to Zt1 compared to Zt5. Probably this is expected

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import subjects
sessions = subjects.nsd.ratNday2+ subjects.nsd.ratSday2+subjects.nsd.ratUday2

In [ ]:
from neuropy.utils.ccg import correlograms

acg_zt1_all = []
acg_zt5_all = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("inter")

    acg_zt = []
    firing_neurons = []
    for t in [post[0],post[0]+4*3600]:
        neurons_t = neurons.time_slice(t, t + 3600)
        n_id = neurons.neuron_ids
        spks = np.concatenate(neurons_t.spiketrains)
        spks_id = np.concatenate(
            [i * np.ones_like(_) for i, _ in zip(n_id, neurons_t.spiketrains)]
        ).astype("int")
        rpl_t = sess.ripple.time_slice(t, t + 3600).flatten()
        spk_bin_ind = np.digitize(spks, rpl_t)
        rpl_indx = spk_bin_ind % 2 == 1
        spks_in_rpl = spks[rpl_indx]
        spks_in_rpl_id = spks_id[rpl_indx]
        sort_ind = np.argsort(spks_in_rpl)
        acg = correlograms(
            spks_in_rpl[sort_ind],
            spks_in_rpl_id[sort_ind],
            sample_rate=30000,
            bin_size=0.001,
            window_size=0.3,
        )
        firing_neurons.append(np.unique(spks_in_rpl_id))
        acg_zt.append(acg)
    _, this, that = np.intersect1d(
        firing_neurons[0], firing_neurons[1], assume_unique=True, return_indices=True
    )

    pairs = np.tril_indices(len(this),k=-1)
    acg_zt = [
        acg_zt[i][np.ix_(ind, ind, np.arange(301))]
        for i, ind in enumerate([this, that])
    ]

    acg_zt = [stats.zscore(_[pairs[0], pairs[1], :],axis=1) for _ in acg_zt]
    nan_indices = np.bitwise_or(*[np.isnan(np.sum(_,axis=1)) for _ in acg_zt])

    acg_zt = [_[~nan_indices,:] for _ in acg_zt]
    acg_zt1_all.append(acg_zt[0])
    acg_zt5_all.append(acg_zt[1])

acg_zt1_all = np.vstack(acg_zt1_all)
acg_zt5_all = np.vstack(acg_zt5_all)

sort_ind = np.argsort(np.argmax(acg_zt1_all,axis=1))

In [ ]:
%matplotlib widget
fig,ax = plt.subplots(1,2)
for i,c in enumerate([acg_zt1_all,acg_zt5_all]):
    ax[i].pcolormesh(c[sort_ind,:])

### Peri SWR spectrogram at selected ZTs for POST
- Basically average wavelet spectrogram across all ripples
- For SD sessions, I did not find any interesting dynamics other than an additional bump in the 15-30 Hz for Zt1, Zt3 and Zt5. Probably similar to what has been reported in Oliva2018 (Origin of gamma frequency power during hippocampal swrs).
- Did not do this analysis for NSD sessions.

In [ ]:
import matplotlib.pyplot as plt
from neuropy.utils.signal_process import WaveletSg
import numpy as np
from neuropy.plotting import Fig
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2
    # + subjects.nsd.ratJday2
    # + subjects.nsd.ratKday2
    # subjects.nsd.ratNday2
    # + subjects.nsd.ratSday2
    # + subjects.nsd.ratRday1
    # + subjects.nsd.ratUday2
    # + subjects.nsd.ratVday1
)
rpl_channels = [39,63,111,95,49,100,85,63,63,36,188,16,99,86]
# rpl_channels=[100]


In [ ]:
from neuropy.utils.signal_process import hilbert_ampltiude_stat
from neuropy.core import Signal

spect = []
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    # rpl_chan = sess.ripple.metadata["channels"][2]

    t_starts = np.arange(0, 5, 2) * 3600 + post[0]

    freqs = np.geomspace(4, 350, 200)
    for t in t_starts:
        signal = sess.eegfile.get_signal(rpl_channels[sub], t, t + 3601)
        rpl_df = sess.ripple.time_slice(t, t + 3600).to_dataframe()
        peakframe = (rpl_df["peaktime"].values * 1250).astype("int")

        rpl_frames = [np.arange(p - 250, p + 250) for p in peakframe]
        rpl_frames = np.concatenate(rpl_frames) - int(t * 1250)
        rpl_frames = rpl_frames[rpl_frames < signal.n_frames]
        new_sig = Signal(
            signal.traces[0][rpl_frames].reshape(1, -1), sampling_rate=1250
        )
        wvlt = WaveletSg(
            signal=new_sig,
            freqs=freqs,
            ncycles=10,
        )
        # wvlt_mean = np.reshape(wvlt.traces, (len(freqs), len(peakframe), -1)).mean(
        #     axis=1
        # )
        wvlt_median = np.median(
            np.reshape(wvlt.traces, (len(freqs), len(peakframe), -1)), axis=1
        )
        spect.append(wvlt_median)


In [ ]:
%matplotlib widget
from neuropy.plotting import Fig
from scipy.ndimage import gaussian_filter
from matplotlib.colors import LogNorm

figure = Fig()
fig, gs = figure.draw(grid=(5, 3))

for i in range(3):
    s = np.dstack(spect[i::3]).mean(axis=-1)
    # s = gaussian_filter(s,sigma=2)
    ax = plt.subplot(gs[i])
    ax.imshow(
        # np.linspace(-200, 200, 500),
        # freqs,
        stats.zscore(s, axis=1),
        # s,
        # extent=[-200,200],
        cmap="jet",
        # shading="gouraud",
        # vmax= 5,
        # norm=LogNorm(),
        interpolation='sinc',
        origin='lower',
        aspect='auto',
    )
    # ax.set_yscale('log')
    ax.set_yticks(np.arange(200)[::30],freqs[::30].round().astype('int'))
    ax.set_xticks([125,250,375],[-100,0,100])
    # ax.plot(np.median(s,axis=1))
# plt.yscale('log')

figure.savefig(subjects.figpath_sd/'ripple_peri_spectrogram_SD')

### Spike triggered raster plot w.r.t ripples during sleep deprivation for individual neurons

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subjects

sessions = subjects.sd.ratSday3

In [ ]:
for sub,sess in enumerate(sessions):
    sd = sess.paradigm['sd']
    rpls = sess.ripple.time_slice(sd[0],sd[1])
    

### Interneuron categorization based on firing rate around ripples start
- using CCG algorithm

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
from neuropy.utils.ccg import correlograms
import subjects

sessions = (
    subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratUday4
    # + subjects.sd.ratVday2
    # + subjects.nsd.ratJday2
    # + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    # + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    # + subjects.nsd.ratVday1
)


In [ ]:
acgs = []
bin_size = 0.01
window_size = 0.2
n_bins = int(window_size // bin_size)
for sub, sess in enumerate(sessions):
    ripple = sess.ripple.to_dataframe().peaktime.values
    neurons = sess.neurons.get_neuron_type("inter")
    for s in neurons.spiketrains:
        combined_s = np.concatenate((s, ripple))
        combined_id = np.concatenate(
            (np.ones(len(s)), 2 * np.ones(len(ripple)))
        ).astype("int")
        sort_ind = np.argsort(combined_s)
        acg = correlograms(
            combined_s[sort_ind],
            combined_id[sort_ind],
            sample_rate=30000,
            bin_size=bin_size,
            window_size=window_size,
        )
        acgs.append(acg[0, 1, :])

acgs = np.asarray(acgs)
t_bin = np.linspace(-n_bins // 2, n_bins // 2, acgs.shape[1])
data = pd.DataFrame(stats.zscore(acgs, axis=1), columns=t_bin)
data["neuron"] = np.arange(acgs.shape[0])
data = pd.melt(
    data,
    id_vars=["neuron"],
    value_vars=t_bin,
    var_name=["time"],
    value_name="frate",
    ignore_index=True,
)

In [ ]:
%matplotlib widget
import seaborn as sns

_,axs = plt.subplots(10,9,sharex=True,sharey=True)
axs = axs.reshape(-1)

acgs_norm = acgs/np.sum(acgs,axis=1,keepdims=True)

for i,a in enumerate(acgs_norm):

    axs[i].fill_between(t_bin,0,a)


# from neuropy.core import Ratemap

# rmap = Ratemap(acgs,np.arange(52),neuron_ids=np.arange(acgs.shape[0]))
# plotting.plot_ratemap(rmap,normalize_tuning_curve=True,pad=2)

# sns.catplot(data=data,x='time',y='frate',col='neuron',col_wrap=5,kind='bar')
# sns.relplot(data=data,x='time',y='frate',col='neuron',col_wrap=5,kind='line')

### Interneurons categorization based on ripple modulation
- The method here is similar to Diba et.al. 2014.
- Here the code that worked and shows some kmeans clustering to identify different categories.
- NOTE: The main has now been ported to ripple_funcs.py in the current directory

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
from neuropy.utils.ccg import correlograms
import subjects

sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratUday4
    # + subjects.sd.ratVday2
    # + subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    # + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    # + subjects.nsd.ratVday1
)


In [ ]:
from sklearn.cluster import KMeans
from ripple_funcs import ripple_modulation

frate_around_rpl = []
for sub, sess in enumerate(sessions):
    neurons = sess.neurons.get_neuron_type("inter")
    n_neurons = len(neurons)
    ripples = sess.ripple.to_dataframe()

    start, peak, stop = sess.ripple.to_dataframe()[
        ["start", "peaktime", "stop"]
    ].values.T
    modulation = ripple_modulation(neurons, start, peak, stop, n_bins=4)
    frate_around_rpl.append(modulation)

frate_around_rpl = np.vstack(frate_around_rpl)
frate_norm = frate_around_rpl / np.sum(frate_around_rpl, axis=1, keepdims=True)
kmeans = KMeans(n_clusters=2, random_state=0).fit(frate_norm)


#### PCA projection of interneurons modulation

In [ ]:
%matplotlib inline
from sklearn.decomposition import PCA
from neuropy import plotting

pca = PCA(n_components=2)
pca.fit(frate_norm)
projected = pca.fit_transform(frate_norm)

# figure = plotting.Fig()
# fig,gs = figure.draw(grid=(1,2))
# gs_ = figure.subplot2grid(gs[1],grid=(2,1))
fig = plt.figure(figsize=(12, 5))
subfigs = fig.subfigures(1, 2, wspace=0.7)
ax_pca = subfigs[0].subplots(1, 1)
ax_pca.set_xlabel("PC1")
ax_pca.set_ylabel("PC2")

hist_figs = subfigs[1].subfigures(2, 1)

colors = ["r", "k"]
for label in [0, 1]:
    indices = kmeans.labels_ == label
    ax_pca.scatter(projected[indices, 0], projected[indices, 1], c=colors[label])

    ind_subset = np.random.choice(np.where(indices)[0], 8)
    frate_subset = frate_norm[ind_subset]
    ax_hists = hist_figs[label].subplots(2, 4, sharex=True, sharey=True).reshape(-1)
    for i, f in enumerate(frate_subset):
        ax_hists[i].stairs(f, np.arange(-8, 9), color=colors[label], fill=True)
        ax_hists[i].set_xticks([-5, 0, 5])
        ax_hists[i].set_xticklabels([-1, 0, 1])

# fig.savefig(subjects.figpath_sd/'interneuron_ripple_classification.pdf')


### Ripple duration comparison between SD and NSD animals
- First, I compared the ripple duration distribution for Zt1 between NSD and SD sessions. Ripple durations were pooled across sessions. The distribution curves looked exactly the same for both groups.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratJday1
    +subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2

    + subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
)

In [ ]:
rpl_dur = pd.DataFrame()
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    t_starts = np.arange(5) * 3600 + post[0]
    # t = post[0]

    # --- for zt1 comparison ------
    # rpl_dur = rpl_dur.append(
    #     pd.DataFrame(
    #         {
    #             "durations": sess.ripple.time_slice(t, t +1* 3600).durations,
    #             "grp": sess.tag,
    #         }
    #     ),
    #     ignore_index=True,
    # )

    # --- for hourly comparison between SD and NSD -----
    rpl_dur = rpl_dur.append(
        pd.concat(
            [
                pd.DataFrame(
                    {
                        "durations": sess.ripple.time_slice(t, t + 3600).durations,
                        "zt": ind + 1,
                        "grp": sess.tag,
                    }
                )
                for ind, t in enumerate(t_starts)
            ]
        ),
        ignore_index=True,
    )


In [ ]:
# %matplotlib widget
import seaborn as sns
from neuropy.plotting import Fig

figure = Fig()
fig, gs = figure.draw(grid=(5, 4))

for i, grp in enumerate(["sd", "nsd"]):
    df = rpl_dur[rpl_dur["grp"] == grp]
    ax = plt.subplot(gs[i])
    ax = sns.histplot(
        data=df,
        x="durations",
        hue="zt",
        stat="probability",
        bins=np.arange(0.05, 0.455, 0.010),
        common_norm=False,
        # cumulative=True,
        element="poly",
        fill=False,
        ax=ax,
        # legend=None,
        palette="Set2",
    )
    ax.set_xscale('log')
    ax.set_xticks([0.1,0.4])
    ax.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())
    ax.set_yscale('log')
    # ax.ticklabel_format(axis='x',style='sci')

# figure.savefig(subjects.figpath_sd/'ripple_duration_distribution')

### Percentage of Cells firing during ripples

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratUday4

    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratUday2
)

In [ ]:
for sub,sess in enumerate(sessions):
    post = sess.paradigm['post'].flatten()
    neurons = sess.neurons.get_neuron_type('pyr')
    rpls = sess.ripple.time_slice(post[0],post[0]+3600)
    modulation = neurons.get_modulation_in_epochs(rpls,n_bins=1).reshape(-1)
    firing_cells = np.where(modulation>0)[0]
    perc = len(firing_cells)/len(modulation)
    print(perc)

### Firing rate within and outside ripples during SD and NSD

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2
    + subjects.sd.ratRday2

    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
)

In [ ]:
from tqdm.notebook import tqdm
from neuropy.core import Epoch

frate_df = []
for sub, sess in enumerate(tqdm(sessions)):
    pre = sess.paradigm["pre"]
    maze = sess.paradigm["maze"]
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(["pyr", "inter"])
    spktrns = neurons.spiketrains

    post_starts = np.arange(9) * 3600 + post[0]
    post_labels = [f"{_}-{_+1}" for _ in range(len(post_starts))]
    # NOTE: subtracted 0.1 to keep the bins monotonically increasing
    post_epochs = Epoch.from_array(post_starts, post_starts + 3600 - 0.1, post_labels)

    all_epochs = pre + maze + post_epochs

    # --- in ripples --------
    rpls = sess.ripple.time_slice(pre.flatten()[0], post[0] + 9 * 3600)
    n_spikes = np.asarray(
        [np.histogram(spk, bins=rpls.flatten())[0] for spk in spktrns]
    )
    in_rpl_spikes = n_spikes[:, ::2]  # individual ripples

    # summing individual ripples within epochs
    in_rpl_spikes_epochs = stats.binned_statistic(
        rpls.starts, in_rpl_spikes, bins=all_epochs.flatten(), statistic="sum"
    )[0][:, ::2]

    n_rpls = np.histogram(rpls.starts, bins=all_epochs.flatten())[0][::2][np.newaxis, :]
    total_rpl_duration_epochs = stats.binned_statistic(
        rpls.starts, rpls.durations, bins=all_epochs.flatten(), statistic="sum"
    )[0][::2]

    # NOTE: divide by n_rpls or divide by total duration ??
    in_rpl_frate_epochs = (
        in_rpl_spikes_epochs / total_rpl_duration_epochs[np.newaxis, :]
    )

    # --- out ripples --------
    n_spikes_epochs = []
    for ep in all_epochs.itertuples():
        n_spikes_epochs.append(neurons.time_slice(ep.start, ep.stop).n_spikes)
    n_spikes_epochs = np.array(n_spikes_epochs).T
    out_rpl_spikes_epochs = n_spikes_epochs - in_rpl_spikes_epochs
    out_rpl_duration_epochs = all_epochs.durations.astype('float') - total_rpl_duration_epochs
    out_rpl_frate_epochs = (
        out_rpl_spikes_epochs / out_rpl_duration_epochs[np.newaxis, :]
    )

    # ----- Normalization ----------
    in_rpl = stats.zscore(in_rpl_frate_epochs, axis=1)
    out_rpl = stats.zscore(out_rpl_frate_epochs, axis=1)

    # baseline = in_rpl[:, 0, np.newaxis]
    # in_rpl = in_rpl - baseline
    # in_rpl = (zt_spikes_in - baseline) / baseline
    assert in_rpl.shape[0] == len(spktrns), "booo"
    assert out_rpl.shape[0] == len(spktrns), "booo hoo"

    in_rpl_df = pd.DataFrame(in_rpl, columns=all_epochs.labels)
    in_rpl_df["where"] = "inside"
    in_rpl_df["neuron_type"] = neurons.neuron_type
    in_rpl_df["grp"] = sess.tag

    out_rpl_df = pd.DataFrame(out_rpl, columns=all_epochs.labels)
    out_rpl_df["where"] = "outside"
    out_rpl_df["neuron_type"] = neurons.neuron_type
    out_rpl_df["grp"] = sess.tag

    frate_sub_df = pd.concat([in_rpl_df, out_rpl_df], axis=0, ignore_index=True)
    frate_df.append(frate_sub_df)

frate_df = pd.concat(frate_df, ignore_index=True)
frate_df = pd.melt(
    frate_df,
    id_vars=["where", "neuron_type", "grp"],
    value_vars=all_epochs.labels,
    var_name="time",
    value_name="frate",
    ignore_index=True,
)

subjects.GroupData().save(frate_df, "frate_in_ripple")


In [ ]:
%matplotlib widget
import seaborn as sns

g = sns.catplot(
    data=frate_df,
    x="Zt",
    y="frate",
    # hue="rpl",
    col="grp",
    kind="bar",
    palette=["#69F0AE", "#FF80AB"],
    ci=95,
    capsize=0.1,
)
axes = g.axes[0]
# [ax.set_yscale('log') for ax in axes]


### Firing rate (scatter plot) within and outside ripples during SD and NSD

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2
    + subjects.sd.ratRday2

    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
)

In [ ]:
from neuropy.core import Epoch
from neuropy.utils.mathutil import min_max_scaler

in_rpl,out_rpl,time =[],[],[]
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")
    spktrns = neurons.spiketrains
    starts = np.arange(9) * 3600 + post[0]

    rpls = sess.ripple.time_slice(post[0],post[0]+9*3600).flatten()

    n_spikes = np.asarray([np.histogram(spk,bins=rpls)[0] for spk in spktrns])
    in_rpl.append(n_spikes[:,::2])
    out_rpl.append(n_spikes[:,1::2])
    time.append(rpls[::2]-post[0])


# frate_df = pd.melt(
#     frate_df,
#     id_vars=["rpl", "grp"],
#     value_vars=np.arange(1,9),
#     var_name="Zt time",
#     value_name="frate",
#     ignore_index=True,
# )


In [ ]:
%matplotlib widget
import seaborn as sns

data = stats.zscore(in_rpl[0],axis=1)
for i in range(97):
    plt.plot(time[0],data[i],'k.')

### PSTH around start of ripples during sleep deprivation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = (
    subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratUday4

    +subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratUday2
)

In [ ]:
from neuropy.core import Epoch

frate_around_rpl,grp_info = [],[]
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")
    grp_info.extend([sess.tag] * len(neurons))

    get_modulation = lambda e: neurons.get_modulation_in_epochs(e, 2)

    hour_modulation = []
    for t in [post[0], post[0] + 4 * 3600]:
        ripples = sess.ripple.time_slice(t, t + 3600)
        start, peak = ripples.to_dataframe()[["start", "peaktime"]].values.T
        start_peak_dur = peak - start
        epochs = Epoch.from_array(start - start_peak_dur, start + start_peak_dur)

        modulation = []
        for s in range(3):
            epoch_slices = epochs[s::3]  # [_[s::3] for _ in (pre_start, start_peak)]
            modulation.append(get_modulation(epoch_slices))
        hour_modulation.append(np.dstack(modulation).sum(axis=2)/ripples.durations.sum())

    frate_around_rpl.append(np.hstack(hour_modulation))

frate_around_rpl = np.vstack(frate_around_rpl)
# frate_norm = frate_around_rpl / np.sum(frate_around_rpl, axis=1, keepdims=True)
frate_norm = stats.zscore(frate_around_rpl, axis=1)

# bins = np.arange(-8, 9)
frate_rpl_df = pd.DataFrame(
    frate_around_rpl, columns=["pre_1h", "post_1h", "pre_5h", "post_5h"]
)
frate_rpl_df["grp"] = grp_info

# frate_session_df = pd.melt(
#     frate_session_df,
#     id_vars=["grp", "label"],
#     value_vars=time_bin,
#     var_name=["time"],
#     value_name="frate",
#     ignore_index=True,
# )


In [ ]:
%matplotlib widget
import seaborn as sns

figure = plotting.Fig()
fig,gs = figure.draw(grid=(2,2))

for i,grp in enumerate(['sd','nsd']):
    df = frate_rpl_df[frate_rpl_df['grp']==grp]

    ax_pre = plt.subplot(gs[i,0])
    sns.scatterplot(data=df ,x='pre_1h',y='pre_5h',ax=ax_pre)
    ax_pre.set_xscale('log')
    ax_pre.set_yscale('log')
    ax_pre.axline((0,0),(1,1))


    ax_post = plt.subplot(gs[i,1])
    sns.scatterplot(data=df ,x='post_1h',y='post_5h',ax=ax_post)
    ax_post.set_xscale('log')
    ax_post.set_yscale('log')
    ax_post.axline((0,0),(1,1))


### Within ripple pairwise correlation comaparison between MAZE vs POST for SD and RS sessions 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import pandas as pd
from scipy import stats
import subjects

sessions = subjects.sd.pf_sess + subjects.nsd.pf_sess

In [ ]:
import pingouin as pg
pcorr_df = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    zt5 = [post[0], post[0] + 5 * 3600]

    rpls_pre = sess.ripple.time_slice(*pre)
    rpls_maze = sess.ripple.time_slice(*maze)
    rpls_zt5 = sess.ripple.time_slice(*zt5)

    neurons = sess.neurons.get_neuron_type("pyr")

    indices = np.tril_indices(len(neurons), k=-1)
    pcorr = lambda e: np.corrcoef(np.hstack(neurons.get_spikes_in_epochs(e)[0]))[
        indices
    ]
    residual= lambda x,y: pg.linear_regression(x,y,remove_na=True).residuals_

    df = pd.DataFrame(
        dict(
            pre=pcorr(rpls_pre),
            maze=pcorr(rpls_maze),
            zt5=pcorr(rpls_zt5),
            grp=sess.tag,
        )
    )
    df.dropna(inplace=True)

    df['maze_residuals'] = residual(df['pre'],df['maze'])
    df['zt5_residuals'] = residual(df['pre'],df['zt5'])
    pcorr_df.append(df)

pcorr_df = pd.concat(pcorr_df, ignore_index=True)


In [ ]:
mat = pcorr_df[pcorr_df['grp']=='sd']
pg.partial_corr(mat,x='maze',y='zt5',covar='pre')

In [ ]:
mat = pcorr_df[pcorr_df['grp']=='nsd']
pg.partial_corr(mat,x='maze',y='zt5',covar='pre')

In [ ]:
%matplotlib widget
import seaborn as sns

sns.lmplot(data=pcorr_df,x='maze_residuals',y='zt5_residuals',col='grp',kind='hex')